# Оценка ответов LLM

### Используемые библиотеки

In [1]:
import numpy as np
import pandas as pd

import re
import string

#Для расчёта метрик
import sacrebleu
from rouge import Rouge 
from bert_score import BERTScorer

from sentence_transformers import SentenceTransformer, util

from tqdm import tqdm
import time

from config import *

#Для отрисовки графиков
import plotly.offline as pyo
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.graph_objects as go
pyo.init_notebook_mode()

In [3]:
class MedQuAD(object):
    '''
        Входные данные:
         * PATH:str: - путь до набора данных MedQuAD.
        Return:
         * refs:list(str): - список обработанных эталонных ответов.
         * resps:list(str): - список обработанных ответов LLM.
    '''
    
    def __init__(self, PATH):
        self.df = pd.read_csv(PATH, sep = ';')
        self.df['gemini-pro'] = self.df['gemini-pro'].astype(str)
        
    #Метод для предобработки ответов LLM
    def preproces_responses(self):
        resp = []
        for index, row in self.df.iterrows():
            resp.append(self._white_space_remove(
                self._remove_articles(self._remove_punc(self._lower(row['gemini-pro'])))))
        
        return resp
    
    #Метод для предобработки эталонных ответов
    def preproces_references(self):
        refs = []
        for index, row in self.df.iterrows():
            refs.append(self._white_space_remove(self._remove_articles(
                self._remove_punc(self._lower(row['references'])))))
        
        return refs
    
    #Метод для удаления артиклей
    def _remove_articles(self, text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    #Метод для удаления лишних пробелов
    def _white_space_remove(self, text):
        return " ".join(text.split())
    
    #Метод для удаления пунктуации
    def _remove_punc(self, text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)
    
    #Метод для приведения к нижнему регистру
    def _lower(self, text):
        return text.lower()

    

### BERTScore
BERTScore - это метрика для оценки качества генерации текста, основанная на использовании предобученной модели BERT (Bidirectional Encoder Representations from Transformers). Она предназначена для оценки сходства между сгенерированным текстом и эталонным текстом с использованием векторных представлений слов, полученных с помощью BERT.

Формула расчета:
BERTScore вычисляется на основе косинусного расстояния между эмбеддингами слов в сгенерированном и эталонном текстах:

$$
BERTScore = {cos(s_i, g) \over N}
$$
$s_i$ - векторное представление слова 

$g$ - векторное представление слова $i$ в эталонном тексте.
$N$ - общее количество слов в сгенерированном тексте.

In [4]:
class BERTScoreMetric(object):
    '''
        Входные данные:
         * refs:list(str): - список обработанных эталонных ответов.
         * resps:list(str): - список обработанных ответов LLM.
        Return:
         * bert_score_f1:list(float): - список метрик BERTScore F1 для каждой пары 
                                    эталонный ответ и ответ LLM.
         * bert_score_r:list(float): - список метрик BERTScore Recall для каждой пары 
                                    эталонный ответ и ответ LLM.
         * bert_score_p:list(float): - список метрик BERTScore Precision для каждой пары 
                                    эталонный ответ и ответ LLM.
    '''
    def __init__(self, refs: list, resps:list):
        self.refs = refs
        self.resps = resps
        self.scorer = BERTScorer(lang = LANG, rescale_with_baseline = True)
    
    def evaluate_metric(self):
        bert_score_f1 = []
        bert_score_r = []
        bert_score_p = []
        for ref, res in tqdm(zip(self.refs, self.resps)):
            P, R, F1 = self.scorer.score([resps],  [[ref]])

            bert_score_f1.append(round(float(F1), 3))
            bert_score_r.append(round(float(R), 3))
            bert_score_p.append(round(float(P), 3))
                             
        return bert_score_f1, bert_score_r, bert_score_p

### Семантическая похожесть на основе Sentence Transformers

Семантическое сходство текстов - это метрика, которая оценивает насколько похожи два текста с точки зрения смысла. 
Ответы модели и эталонные значения кодируются в эмбеддинги с помощью одной из предварительно обученных моделей "all-MiniLM-L6-v2" SentenceTransformer. 

Затем оценивается косинусное сходство ответа и эталонного значения 
Косинусное сходсттво - это мера сходства между двумя ненулевыми векторами, определенными в пространстве внутреннего произведения.
Косинусное сходство принадлежит интервалу [0,1] - два пропорциональных вектора имеют косинусное сходство, равное 1, а два ортогональных вектора имеют сходство, равное 0.

In [5]:
class SemanticSimilarityMetric(object):
    '''
        Входные данные:
         * refs:list(str): - список обработанных эталонных ответов.
         * resps:list(str): - список обработанных ответов LLM.
        Return:
         * sem_sim:list(float): - список метрик косинусное сходство для каждой пары 
                                эталонный ответ и ответ LLM.
    '''
    def __init__(self, refs: list, resps:list):
        self.refs = refs
        self.resps = resps
        self.model = SentenceTransformer(MODEL)
    
    def evaluate_metric(self):
        sem_sim = []

        for ref, res in tqdm(zip(self.refs, self.resps)):
            #Построение эмбеддингов с помощью предобученной модели all-MiniLM-L6-v2
            reference_embedding = self.model.encode(ref, convert_to_tensor = True)
            response_embedding = self.model.encode(res, convert_to_tensor = True)
            #Расчет косинусного сходства
            cosine_score = util.cos_sim(reference_embedding, response_embedding)
            sem_sim.append(round(float(cosine_score), 3))
                             
        return sem_sim

### ROUGE

ROUGE метрика вычисляется на основе пересечения (overlap) между n-граммами (конкретно, униграммами, биграммами и т. д.) в сгенерированном суммаризированном тексте и эталонном суммаризированном тексте. Она имеет несколько вариантов, таких как ROUGE-N, ROUGE-L и ROUGE-W.

ROUGE-N: Вычисляет пересечение униграмм, биграмм, триграмм и т.д. между сгенерированным и эталонным текстами.

ROUGE-L: Вычисляет самую длинную общую подпоследовательность (longest common subsequence) между сгенерированным и эталонным текстами.

ROUGE-W: Вычисляет пересечение между взвешенными n-граммами с учетом позиции слов в предложении.

Формула расчета метрики ROUGE-N выглядит следующим образом:
$$
ROUGE_N = {\sum_r \sum_n count_m (Ngram) \over \sum_r \sum_n count_t (Ngram)}
$$
$N$ - максимальная длина n-граммы (обычно используется до 4).

$count_m (Ngram)$ - количество n-грамм, которые встречаются в сгенерированном тексте и в эталонных текстах.

$count_t (Ngram)$ - общее количество n-грамм в эталонных текстах.

In [6]:
class RougeMetric(object):
    '''
        Входные данные:
         * refs:list(str): - список обработанных эталонных ответов.
         * resps:list(str): - список обработанных ответов LLM.
        Return:
         * rouge_1:list(float): - список метрик Rouge-1 F1 для каждой пары 
                                    эталонный ответ и ответ LLM.
         * rouge_2:list(float): - список метрик Rouge-2 F1 для каждой пары 
                                    эталонный ответ и ответ LLM.
         * rouge_l:list(float): - список метрик Rouge-L для каждой пары 
                                    эталонный ответ и ответ LLM.
    '''
    def __init__(self,  refs: list, resps:list):
        self.refs = refs
        self.resps = resps
        self.rouge = Rouge()
    
    def evaluate_metric(self):
        rouge_1 = []
        rouge_2 = []
        rouge_l = []
        
        for ref, res in tqdm(zip(self.refs, self.resps)):
            scores = self.rouge.get_scores(res, ref)
            rouge_1.append(round(scores[0]['rouge-1']['f'], 3))
            rouge_2.append(round(scores[0]['rouge-2']['f'], 3))
            rouge_l.append(round(scores[0]['rouge-l']['f'], 3))
                             
        return rouge_1, rouge_2, rouge_l

### BLEU

BLEU (Bilingual Evaluation Understudy) - это метрика, которая используется для оценки качества машинного перевода путем сравнения автоматически сгенерированных переводов с эталонными (человеческими) переводами. 

BLEU метрика рассчитывается на основе точности (precision) n-грамм в сгенерированном переводе:

$$
BLEU = BP * exp(\sum_{n=1}^N w_n log(p_n))
$$

$BP$ - штраф за близость перевода, для того чтобы уменьшить влияние коротких переводов.

$p_n$- точность n-грамм.

$w_n$ - весовой коэффициент для каждой n-граммы.

$N$ - максимальная длина n-граммы (обычно используется до 4).

In [7]:
class BLEUMetric(object):
    '''
        Входные данные:
         * refs:list(str): - список обработанных эталонных ответов.
         * resps:list(str): - список обработанных ответов LLM.
        Return:
         * bleu:list(float): - список метрик BLEU для каждой пары 
                                    эталонный ответ и ответ LLM.
         * ref_len:list(int): - длина эталонного ответа, полученна при расчете метрики BLEU.
         * resp_len:list(int): - длина ответа LLM, полученна при расчете метрики BLEU.
    '''
    
    def __init__(self, refs: list, resps:list):
        self.refs = refs
        self.resps = resps
    
    def evaluate_metric(self):
        bleu = []
        ref_len = []
        resp_len = []
        
        for ref, res in tqdm(zip(self.refs, self.resps)):
            scores = sacrebleu.sentence_bleu(res, [ref])
            bleu.append(round(scores.score/100, 3))
            ref_len.append(scores.ref_len)
            resp_len.append(scores.sys_len)
                             
        return bleu, ref_len , resp_len 

In [8]:
class FailuresMetric(object):
    '''
        Входные данные:
         * refs:list(str): - список обработанных эталонных ответов.
         * resps:list(str): - список обработанных ответов LLM.
        Return:
         * failures:list(int): - количество раз, когда произошел сбой и не было дано ответа LLM.
    '''
    
    def __init__(self, refs: list, resps:list):
        self.refs = refs
        self.resps = resps
    
    def evaluate_metric(self):
        failures = []
        
        for res in tqdm(self.resps):
            if res == 'nan':
                failures.append(1)
            else:
                failures.append(0)
                             
        return failures

In [9]:
class LLMEvaluation(object):
    '''
        Входные данные:
         * model_name:str: - название модели LLM.
         * bleu:bool: - булево значение, необлодимо ли использовать метрику BLEU.
         * rouge:bool: - булево значение, необлодимо ли использовать метрику ROUGE.
         * bertscore:bool: - булево значение, необлодимо ли использовать метрику BERTScore.
         * sem_similarity:bool: - булево значение, необлодимо ли использовать метрику семантического сходства.
         * failures_resp:bool: - булево значение, необлодимо ли использовать метрику failures.
         
         Return:
         * table:pd.DataFrame: - агрегированный набор данных, который содержит средние значения по метрикам.
         * df:pd.DataFrame: - набор данных, который содержит метрики для каждой пары 
                                    эталонного ответа и ответа LLM.
    '''
    
    def __init__(self, 
                 model_name: str,
                 bleu: bool, 
                 rouge: bool,
                 bertscore: bool,
                 sem_similarity: bool,
                 failures_resp: bool,
                ):
        
        self.model_name = model_name
        self.df = pd.read_csv(PATH, sep = ';')
        self.refs = MedQuAD(PATH).preproces_references()
        self.resps = MedQuAD(PATH).preproces_responses()
        self.bleu = bleu
        self.rouge = rouge
        self.bertscore = bertscore
        self.sem_similarity = sem_similarity
        self.failures_resp = failures_resp
          
            
    def evaluate(self):
        if self.failures_resp:
            fm = FailuresMetric(self.refs, self.resps).evaluate_metric()
            self.df['failures'] = fm
        if self.bleu:
            bm = BLEUMetric(self.refs, self.resps).evaluate_metric()
            self.df['bleu'] = bm[0]
            self.df['ref_len'] = bm[1]
            self.df['resp_len'] = bm[2]
        if self.rouge:
            rgm = RougeMetric(self.refs, self.resps).evaluate_metric()
            self.df['rouge_1'] = rgm[0]
            self.df['rouge_2'] = rgm[1]
            self.df['rouge_l'] = rgm[2]
        if self.bertscore:
            bsm = BERTScoreMetric(self.refs, self.resps).evaluate_metric()
            self.df['bert_score_f1'] = bsm[0]
            self.df['bert_score_r'] = bsm[1]
            self.df['bert_score_p'] = bsm[2]
        if self.sem_similarity:
            ssm = SemanticSimilarityMetric(self.refs, self.resps).evaluate_metric()
            self.df['sem_sim'] = ssm

        self.df.to_csv(OUTPUT_FILE, sep = ';', index = False)
        return self.df
    
    def agregate_metrics(self):
        if self.failures_resp:
            self.df = self.df[self.df['failures'] == 0]
        
        table = self.df.groupby(['model_name'], as_index=False)[self.df.columns[7:]].agg(['mean']).round(3).reset_index()
        table.columns = table.columns.droplevel(1)
        table.columns = table.columns
        
        table.to_csv(OUTPUT_FILE_STAT, sep = ';', index = False)
        return table
        
        
        
    

In [10]:
class PlotMetric(object):
    '''
        Входные данные:
         * table:pd.DataFrame: - агрегированный набор данных, который содержит средние значения по метрикам.
         * df:pd.DataFrame: - набор данных, который содержит метрики для каждой пары 
                                    эталонного ответа и ответа LLM.
         * metric:str: - название метрики, для которой строится график.
         * model:list(str): - список названий моделей LLM.
                                    
         Return:
         * fig:plotly.type: - графики plotly для отображения распределения метрик, 
                             таблицы средних значенйи и диаграммы bar.

    '''
    
    def __init__(self, df, metric, models, table):
        self.df = df
        self.table = table
        self.metric = metric
        self.models = models
        
    def rouge_distribution(self):
        group_labels = ['rouge-1', 'rouge-2',  'rouge-l']

        colors = ['rgb(0, 0, 100)', 'rgb(0, 200, 200)', 'rgb(171, 28, 28)']

        fig = ff.create_distplot(
            [self.df['rouge_1'], self.df['rouge_2'], self.df['rouge_l']], group_labels,
            bin_size=.2, colors=colors, show_hist=False)

        r1 = np.mean(self.df['rouge_1'])
        r2 = np.mean(self.df['rouge_2'])
        rl = np.mean(self.df['rouge_l'])
        fig.add_shape(type = "line",x0 = r1, x1 = r1, y0 = 0, y1 = 10 , xref='x', yref='y',
                       line = dict(color = 'rgb(0, 0, 100)', dash = 'dash'))
        fig.add_shape(type = "line",x0 = r2, x1 = r2, y0 = 0, y1 = 10 , xref='x', yref='y',
                       line = dict(color = 'rgb(0, 200, 200)', dash = 'dash'))
        fig.add_shape(type = "line",x0 = rl, x1=rl, y0 = 0, y1 = 10 , xref='x', yref='y',
                       line = dict(color = 'rgb(171, 28, 28)', dash = 'dash'))
        fig.update_layout(title_text='ROUGE-N')

        return fig
    
    def metric_distribution(self):

        colors = ['rgb(0, 0, 100)']

        fig = ff.create_distplot(
            [self.df[self.metric]], [self.metric],
            bin_size=.2, colors=colors, show_hist=False)

        m = np.mean(self.df[self.metric])
        med = np.median(self.df[self.metric])
        
        fig.add_shape(type = "line", x0 = m, x1 = m, y0 = 0, y1 = 10 , xref='x', yref='y',
                       line = dict(color = 'rgb(0, 0, 100)', dash = 'dash'))
        
        fig.add_shape(type = "line", x0 = med, x1 = med, y0 = 0, y1 = 10 , xref='x', yref='y',
                       line = dict(color = 'rgb(171, 28, 28)', dash = 'dash'))
        fig.update_layout(title_text = self.metric)

        return fig
        
        
    def plot_report(self):
        # Инициализация фигуры ff.create_table(table)
        fig = ff.create_table(table, height_constant = 15)

        models = ['Baseline'] + self.models
        if 'len' in self.metric:
            mean_metric = [100] + [self.table[self.metric].iloc[0]]
        else:
            mean_metric = [1] + [self.table[self.metric].iloc[0]]

        trace1 = go.Bar(x=[models[0]], y=[mean_metric[0]], xaxis='x2', yaxis='y2',
                        marker=dict(color='gray'),
                        name='Baseline')
        trace2 = go.Bar(x=[models[1]], y=[mean_metric[1]], xaxis='x2', yaxis='y2',
                        marker=dict(color='#0099ff'),
                        name=self.models[0])


        fig.add_traces([trace1, trace2])

        fig['layout']['xaxis2'] = {}
        fig['layout']['yaxis2'] = {}


        fig.layout.yaxis.update({'domain': [0, .45]})
        fig.layout.yaxis2.update({'domain': [.6, 1]})


        fig.layout.yaxis2.update({'anchor': 'x2'})
        fig.layout.xaxis2.update({'anchor': 'y2'})
        fig.layout.yaxis2.update({'title': self.metric})


        fig.layout.margin.update({'t':70, 'l':50})
        fig.layout.update({'title': 'LLM Evaluation'})


        fig.layout.update({'height':600})
        return fig
        

In [12]:
%%time
metrics = LLMEvaluation('gemini-pro', True, True, False, True, True)
mall = metrics.evaluate()

100%|█| 1007/1007 [00:00<00:00, 747551.
1007it [00:00, 1160.76it/s]
1007it [00:11, 89.68it/s]
1007it [02:30,  6.68it/s]


CPU times: user 2min 13s, sys: 10.8 s, total: 2min 24s
Wall time: 2min 44s


In [13]:
table = metrics.agregate_metrics()

In [17]:
df = pd.read_csv('datasets/golden_dataset_metrics.csv', sep = ';')

In [18]:
PlotMetric(df, 'sem_sim', ['gemini-pro'], table).plot_report().show()

In [19]:
PlotMetric(df, 'rouge_1', ['gemini-pro'], table).rouge_distribution().show()

In [20]:
PlotMetric(df, 'rouge_1', ['gemini-pro'], table).plot_report().show()

In [21]:
PlotMetric(df, 'cosine_sim', ['gemini-pro'], table).metric_distribution().show()

In [22]:
df.head()

,id_question,model_name,qtype,question,references,gemini-pro,bleu,ref_len,resp_len,rouge_1,rouge_2,rouge_l,cosine_sim,bert_score_f1,bert_score_r,bert_score_p
0,1,gemini-pro,frequency,How many people are affected by Stormorken syn...,Stormorken syndrome is a rare disorder. Approx...,Stormorken syndrome is an extremely rare genet...,0.029,14.0,58.0,0.237,0.060,0.203,0.817,0.234,0.397,0.079
1,2,gemini-pro,inheritance,Is PDGFRB-associated chronic eosinophilic leuk...,PDGFRB-associated chronic eosinophilic leukemi...,PDGFRB-associated chronic eosinophilic leukemi...,0.038,65.0,106.0,0.250,0.072,0.203,0.798,0.155,0.200,0.107
2,3,gemini-pro,inheritance,Is pulmonary arterial hypertension inherited ?,Pulmonary arterial hypertension is usually spo...,Pulmonary arterial hypertension (PAH) can be i...,0.036,117.0,105.0,0.392,0.080,0.224,0.821,0.068,0.057,0.076
3,4,gemini-pro,research,what research (or clinical trials) is being do...,NINDS conducts and supports research on disord...,Research efforts for spinal cord infarction en...,0.019,30.0,123.0,0.162,0.042,0.126,0.503,-0.037,0.111,-0.180
4,5,gemini-pro,frequency,How many people are affected by Proteus syndro...,Proteus syndrome is a rare condition with an i...,Proteus syndrome is an extremely rare genetic ...,0.039,73.0,98.0,0.323,0.073,0.246,0.885,0.189,0.253,0.125
